<a href="https://colab.research.google.com/github/katie0809/2022Military-Chatbot-jupyter/blob/main/source/22'Military_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Version History

|version|Score|Changes|Comment|
|-|-|-|-|
|1.0|-|Initial model| 22.07.13 |

## Environments

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
API_URL_SAMPLE='https://openapi.mnd.go.kr/3232313634343937343732313632313430/xml/DS_MND_ENLSTMN_DCNT_BEF_INF/1/5/'
HOME_PATH='/content/drive/MyDrive'
WORK_DIR=HOME_PATH+'/Colab Notebooks'
FILES_DIR='/assets'
MODEL_DIR=WORK_DIR+'/models'

In [3]:
%cd $WORK_DIR

/content/drive/MyDrive/Colab Notebooks


In [4]:
!python3 --version

Python 3.7.13


## Mecab 설치 
:한국어 형태소분석기 

In [5]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

--2022-07-20 12:30:32--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2022-07-20 12:30:33--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc006cc74b0fd5ca8d4f637d7b40.dl.dropboxusercontent.com/cd/0/get/BpbP26JdLiJULNMwuSPK5XMucEEIBQKoSwYfjlNwyCOBZZxClq6Q9x9qDYEuDag8L0kB87YTJq-6PW5l-8sY7tNQfxyY4DHxLJNM9XHb44ZLWEEWPzxsSJ-y6lf2TrMn1AeVdk-IUwR7HBh7o-7mdklxdLFCHbwS-0-M6XQhfTQpuIIaaDGnqdxWObS9KAB6sZk/file?dl=1# [following]
--2022-07-20 12:30:33--  https://uc006cc74b0fd5ca8d4f637d7b40.dl.dropboxusercontent.com/cd/0/g

#### 동작 확인 

In [6]:
# 정상 동작 확인
from konlpy.tag import Okt, Mecab

okt = Okt()
mecab = Mecab()

In [7]:
okt.morphs('한글 형태소 분석기(오케이티)로 테스트를 해보았습니다. 정상 설치 및 동작이 잘 됩니다.')

['한글',
 '형태소',
 '분석',
 '기',
 '(',
 '오',
 '케이티',
 ')',
 '로',
 '테스트',
 '를',
 '해보았습니다',
 '.',
 '정상',
 '설치',
 '및',
 '동작',
 '이',
 '잘',
 '됩니다',
 '.']

In [8]:
mecab.morphs('한글 형태소 분석기(미켑)로 테스트를 해보았습니다. 정상 설치 및 동작이 잘 됩니다.')

['한글',
 '형태소',
 '분석기',
 '(',
 '미',
 '켑',
 ')',
 '로',
 '테스트',
 '를',
 '해',
 '보',
 '았',
 '습니다',
 '.',
 '정상',
 '설치',
 '및',
 '동작',
 '이',
 '잘',
 '됩니다',
 '.']

### Mecab 사용자 사전 추가 
- 예시 : '모바일서비스팀' 사용자 사전 추가 

In [10]:
mecab.morphs('코스콤 모바일서비스팀은 맨날 야근한다.')

['코스콤', '모바일', '서비스', '팀', '은', '맨날', '야근', '한다', '.']

In [13]:
# mecab-ko-dic이 다운되어있는 위치로 이동 
%cd /tmp/mecab-ko-dic-2.1.1-20180720/
!ls

/tmp/mecab-ko-dic-2.1.1-20180720
aclocal.m4	EF.csv	     Makefile	     NR.csv		user-dic
AUTHORS		EP.csv	     Makefile.am     Person-actor.csv	VA.csv
autogen.sh	ETM.csv      Makefile.in     Person.csv		VCN.csv
autom4te.cache	ETN.csv      matrix.bin      Place-address.csv	VCP.csv
ChangeLog	feature.def  matrix.def      Place.csv		VV.csv
char.bin	Foreign.csv  missing	     Place-station.csv	VX.csv
char.def	Group.csv    MM.csv	     pos-id.def		Wikipedia.csv
clean		Hanja.csv    model.bin	     Preanalysis.csv	XPN.csv
CoinedWord.csv	IC.csv	     model.def	     README		XR.csv
config.log	Inflect.csv  NEWS	     rewrite.def	XSA.csv
config.status	INSTALL      NNBC.csv	     right-id.def	XSN.csv
configure	install-sh   NNB.csv	     Symbol.csv		XSV.csv
configure.ac	J.csv	     NNG.csv	     sys.dic
COPYING		left-id.def  NNP.csv	     tools
dicrc		MAG.csv      NorthKorea.csv  unk.def
EC.csv		MAJ.csv      NP.csv	     unk.dic


In [14]:
# 미리 만들어둔 사용자 사전 nnp파일 read
with open(WORK_DIR+"/nnp.csv", 'r', encoding='utf-8') as f:
  file_new = f.readlines()
file_new

['대우,,,,NNP,*,F,대우,*,*,*,*,*\n',
 '구글,,,,NNP,*,T,구글,*,*,*,*,*\n',
 '모바일서비스팀,,,,NNP,*,F,모바일서비스팀,*,*,*,*,*']

In [25]:
# 기존 /user-dic/nnp.csv 파일에 내용 추가한다 
with open("./user-dic/nnp.csv", 'w', encoding='utf-8') as f:
  for line in file_new:
    f.write(line)

%cat ./user-dic/nnp.csv

대우,,,,NNP,*,F,대우,*,*,*,*,*
구글,,,,NNP,*,T,구글,*,*,*,*,*
모바일서비스팀,,,,NNP,*,F,모바일서비스팀,*,*,*,*,*

In [27]:
# mecab의 tool 사용해 추가한 내용으로 다시 빌드
!bash ./tools/add-userdic.sh

generating userdic...
nnp.csv
/tmp/mecab-ko-dic-2.1.1-20180720/tools/../model.def is not a binary model. reopen it as text mode...
reading /tmp/mecab-ko-dic-2.1.1-20180720/tools/../user-dic/nnp.csv ... 
done!
person.csv
/tmp/mecab-ko-dic-2.1.1-20180720/tools/../model.def is not a binary model. reopen it as text mode...
reading /tmp/mecab-ko-dic-2.1.1-20180720/tools/../user-dic/person.csv ... 
done!
place.csv
/tmp/mecab-ko-dic-2.1.1-20180720/tools/../model.def is not a binary model. reopen it as text mode...
reading /tmp/mecab-ko-dic-2.1.1-20180720/tools/../user-dic/place.csv ... 
done!
test -z "model.bin matrix.bin char.bin sys.dic unk.dic" || rm -f model.bin matrix.bin char.bin sys.dic unk.dic
/usr/local/libexec/mecab/mecab-dict-index -d . -o . -f UTF-8 -t UTF-8
reading ./unk.def ... 13
emitting double-array: 100% |###########################################| 
reading ./Person.csv ... 196459
reading ./VX.csv ... 125
reading ./CoinedWord.csv ... 148
reading ./J.csv ... 416
reading ./MM

In [31]:
!make install

make[1]: Entering directory '/tmp/mecab-ko-dic-2.1.1-20180720'
make[1]: Nothing to be done for 'install-exec-am'.
 /bin/mkdir -p '/usr/local/lib/mecab/dic/mecab-ko-dic'
 /usr/bin/install -c -m 644 model.bin matrix.bin char.bin sys.dic unk.dic left-id.def right-id.def rewrite.def pos-id.def dicrc '/usr/local/lib/mecab/dic/mecab-ko-dic'
make[1]: Leaving directory '/tmp/mecab-ko-dic-2.1.1-20180720'


In [32]:
# 사용자 사전은 적용됐지만 단어 분해시 우선순위가 낮아서 원하는 형태로 적용되지 않음 
# /tmp/mecab-ko-dic-2.1.1-20180720/tools# ./mecab-bestn.sh 
# #표현층,품사,의미부류,좌문맥ID,우문맥ID,낱말비용,연접비용,누적비용
# 모바일서비스팀 
# 모바일서비스팀  NNP,,1786,3545,2953,-1885,1068
# EOS
# 모바일  NNP,,1786,3546,2953,-1885,1068
# 서비스  NNG,,1780,3533,1916,281,3265
# 팀      NNG,,1780,3534,2188,269,5722
# EOS

# !bash ./tools/mecab-bestn.sh
mecab.morphs('코스콤 모바일서비스팀은 맨날 야근한다.')

['코스콤', '모바일', '서비스', '팀', '은', '맨날', '야근', '한다', '.']

In [33]:
# 우선순위 조정(단어비용 낮추기) => https://mondayus.tistory.com/46
!sudo make clean
!sudo make install

대우,,,,NNP,*,F,대우,*,*,*,*,*

구글,,,,NNP,*,T,구글,*,*,*,*,*

모바일서비스팀,,,,NNP,*,F,모바일서비스팀,*,*,*,*,*


In [34]:
# 결과확인 
# 모바일서비스팀 
# 모바일서비스팀  NNP,,1786,3545,0,-1885,-1885
# EOS
# 모바일  NNP,,1786,3546,2953,-1885,1068
# 서비스  NNG,,1780,3533,1916,281,3265
# 팀      NNG,,1780,3534,2188,269,5722
# EOS
mecab.morphs('코스콤 모바일서비스팀은 맨날 야근한다.')

['코스콤', '모바일', '서비스', '팀', '은', '맨날', '야근', '한다', '.']

#### FastText
: https://pypi.org/project/fasttext/

In [ ]:
# !git clone https://github.com/facebookresearch/fastText.git
%cd $WORK_DIR
# !wget https://files.pythonhosted.org/packages/a4/86/ff826211bc9e28d4c371668b30b4b2c38a09127e5e73017b1c0cd52f9dfa/fasttext-0.8.3.tar.gz
# !tar -xvf fasttext-0.8.3.tar.gz

/content/drive/MyDrive/Colab Notebooks


In [ ]:
%cd $WORK_DIR/fasttext-0.8.3
!pip install .

/content/drive/MyDrive/Colab Notebooks/fasttext-0.8.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/drive/MyDrive/Colab Notebooks/fasttext-0.8.3
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for fasttext: filename=fasttext-0.8.3-cp37-cp37m-linux_x86_64.whl size=1090234 sha256=6bf9cdee02f3fbfc4fcc4ee8b86618769cb1cc22bec5b3490e0884e82728d358
  Stored in directory: /root/.cache/pip/wheels/04/f5/eb/01dae2ec6c6262cc2e650bb47ddb89100623ad833336ad2568
Successfully built fasttext


#### 3i4k 한국어 문장 의도분류모델

In [ ]:
# 소스 다운로드
%cd $WORK_DIR
# !git clone https://github.com/warnikchow/3i4k.git

/content/drive/MyDrive/Colab Notebooks


In [ ]:
# 3i4k 의존모듈 설치 
%cd $WORK_DIR/3i4k
!pip install -r Requirements.txt

/content/drive/MyDrive/Colab Notebooks/3i4k
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 304 kB 5.1 MB/s 
     |████████████████████████████████| 1.4 MB 77.5 MB/s 
     |████████████████████████████████| 4.9 MB 81.4 MB/s 
     |████████████████████████████████| 1.6 MB 70.2 MB/s 
ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==1.4.1 (from versions: 1.13.1, 1.13.2, 1.14.0, 1.15.0, 1.15.2, 1.15.3, 1.15.4, 1.15.5, 2.0.0, 2.0.1, 2.0.2, 2.0.3, 2.0.4, 2.1.0, 2.1.1, 2.1.2, 2.1.3, 2.1.4, 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1)
ERROR: No matching distribution found for tensorflow-gpu==1.4.1


In [ ]:
# Tensorflow 버전 하향
!pip uninstall tensorflow
!pip install tensorflow==1.15

Found existing installation: tensorflow 2.8.2+zzzcolab20220527125636
Uninstalling tensorflow-2.8.2+zzzcolab20220527125636:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.8.2+zzzcolab20220527125636.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.8.2+zzzcolab20220527125636
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 412.3 MB 21 kB/s 
     |████████████████████████████████| 503 kB 76.6 MB/s 
     |████████████████████████████████| 50 kB 8.2 MB/s 
     |████████████████████████████████| 3.8 MB 75.2 MB/s 
  Created wheel 

In [ ]:
!pip install Keras==2.1.2
!pip install nltk==3.3
!pip install librosa==0.6.1
!pip3 install numba==0.48.0
!pip install h5py==2.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.9 MB 5.0 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220527125636 requires keras<2.9,>=2.8.0rc0, but you have keras 2.1.2 which is incompatible.
kapre 0.3.

In [ ]:
# Word Vectors 다운로드 
# %cd $WORK_DIR/3i4k
# !mkdir vectors
# %cd vectors
# !wget -O model_drama.bin --no-check-certificate "https://drive.google.com/file/d/1jHbjOcnaLourFzNuP47yGQVhBTq6Wgor"
# !unzip vectors.zip

In [ ]:
%cd $WORK_DIR/3i4k
!python3 3i4k_demo.py

/content/drive/MyDrive/Colab Notebooks/3i4k












#########################################################
#                                                       #
#       Demonstration video: 3i for Korean (3i4K)       #
#                                                       #
#########################################################
Using TensorFlow backend.


2022-07-19 10:16:19.545749: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2022-07-19 10:16:19.550074: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200220000 Hz
2022-07-19 10:16:19.550659: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x47178bc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-07-19 10:16:19.550690: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-07-19 

In [ ]:
from classify import pred_only_text

In [ ]:
print(pred_only_text('배고프다 점심 뭐먹지?'))

# Install Libraries

# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns

import sklearn

import tensorflow as tf
from tensorflow import keras

In [ ]:
print(tf.__version__)
print(sklearn.__version__)
print(np.__version__)
print(pd.__version__)

2.8.2
1.0.2
1.21.6
1.3.5


# Load Files

In [ ]:
df = pd.read_csv(WORK_DIR+'/sample.csv', encoding='cp949')
df

,행번호,지역,시설명,할인행사명,시작일,종료일,연락처,홈페이지,상세설명
0,1,서울,세종호텔,연중 객실할인,2018-09-17,NaN,02-3705-9115,https://www.welfare.mil.kr,"객실: 50% 이상,연회 : 10%15%"
1,2,경남,거제삼성호텔,연중 객실할인,2017-01-01,NaN,055-631-2114,https://www.welfare.mil.kr,객실 55%62%내외
2,3,경기,베어스타운리조트,연중 객실할인,2016-11-25,NaN,02-456-2014,https://www.cielsportsmedia.co.kr,"프로모션(패키지상품)위주로 운영, 현장결재(최저가보장), 씨엘스포츠 폐쇄몰 이용"
3,4,강원,하이원리조트,연중 객실할인,2017-06-19,NaN,1588-7789,https://www.welfare.mil.kr,"프로모션(패키지상품)위주로 운영, 현장결재(최저가보장)"
4,5,강원,용평리조트 객실할인,연중 객실할인,2016-07-01,NaN,02-456-2014,https://www.welfare.mil.kr,"프로모션(패키지상품)위주로 운영, 현장결재(최저가보장),씨엘스포츠 폐쇄몰 이용"
...,...,...,...,...,...,...,...,...,...
84,85,전국,제주항공(수송사),연중 운임할인,2016-12-02,2026-06-30,02-3419-7553,https://www.welfare.mil.kr,7~25%
85,86,전국,통인익스프레스,이용할인,2018-02-01,2023-02-28,1899-0123,https://www.welfare.mil.kr,12%20%
86,87,전국,로젠이사,이용할인,2018-02-01,2023-02-28,1544-0024,https://www.welfare.mil.kr,7%13%
87,88,전국,대신로지스틱스,이용할인,2018-02-01,2023-02-28,02-3436-0123,https://www.welfare.mil.kr,5%7%


In [ ]:
word_dict = pd.read_csv(WORK_DIR+'/word_dictionary.csv', encoding='cp949')
word_dict

,단어 명,물리 명,영문 명,금칙어,분류어 여부,정의내용
0,5,5,FIVE,NaN,NaN,생략
1,12,12,TWELVE,NaN,NaN,생략
2,13,13,THIRTEEN,NaN,NaN,생략
3,14,14,FOURTEEN,NaN,NaN,생략
4,15,15,FIFTEEN,NaN,NaN,생략
...,...,...,...,...,...,...
18345,희생공헌자,SCRCTBR,SACRIFICE CONTRIBUTOR,NaN,NaN,생략
18346,희생자,VICTIM,VICTIM,NaN,NaN,생략
18347,희생자심사,VCJDG,VICTIM JUDGING,NaN,NaN,생략
18348,희소,RARN,RARENESS,NaN,NaN,생략
